In [1]:
import ccxt
from crypto_data_fetcher.bybit import BybitFetcher
import pandas as pd

In [2]:
bybit = ccxt.bybit()
fetcher = BybitFetcher(ccxt_client=bybit)

df = fetcher.fetch_ohlcv(
    market='BTCUSDT', # 市場のシンボルを指定。この場合は https://ftx.com/trade/BTC-PERP を取得
    interval_sec=1 * 60, # 足の間隔を秒単位で指定。この場合は5分足
)

display(df)

,op,hi,lo,cl,volume
timestamp,,,,,
2020-03-25 06:37:25+00:00,6500.0,6500.0,6500.0,6500.0,0.000
2020-03-25 10:36:00+00:00,6500.0,6500.0,6500.0,6500.0,0.001
2020-03-25 10:37:00+00:00,6500.0,6500.0,6500.0,6500.0,0.000
2020-03-25 10:38:00+00:00,6500.0,6500.0,6500.0,6500.0,0.000
2020-03-25 10:39:00+00:00,6500.0,6500.0,6500.0,6500.0,0.000
...,...,...,...,...,...
2022-01-01 15:07:00+00:00,46972.5,46972.5,46972.0,46972.0,5.794
2022-01-01 15:08:00+00:00,46972.0,46972.5,46972.0,46972.5,17.241
2022-01-01 15:09:00+00:00,46972.5,46972.5,46950.0,46950.0,8.212


In [4]:
d2 = df[df.index >= '2021-12-01 00:00:00']

In [ ]:
d2.to_csv('bybit_